<a href="https://colab.research.google.com/github/satyamverma95/DL_NLP/blob/main/Pre_requisite_finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [2]:
#!pip install evaluate
#!pip install huggingface_hub
#!pip install datasets
#!pip install transformers==4.28.0
#!pip install --upgrade accelerate

Getting Token authentication cell and input your username and password:

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
import transformers

print(transformers.__version__)


4.28.0


# Fine-tuning a model on a text classification task

This notebook is built to run on any of the tasks in the list above, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a classification head. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [5]:
from datasets import  load_metric

task = "rte"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16
metric = load_metric('glue', task)

<ipython-input-5-6da0497477ac>:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', task)


The DistilBERT model has several pre-trained variants with different configurations. As of my knowledge cutoff in September 2021, the following DistilBERT models were available:

1. `distilbert-base-uncased`: The base variant of DistilBERT with an uncased vocabulary.
2. `distilbert-base-uncased-distilled-squad`: A variant of DistilBERT fine-tuned on the Stanford Question Answering Dataset (SQuAD). It is specifically trained for question answering tasks.
3. `distilbert-base-cased`: Similar to `distilbert-base-uncased`, but with a cased vocabulary.
4. `distilbert-base-cased-distilled-squad`: Similar to `distilbert-base-uncased-distilled-squad`, but with a cased vocabulary.
5. `distilbert-base-multilingual-cased`: A variant of DistilBERT trained on a multilingual corpus with a cased vocabulary. It supports multiple languages.
6. `distilbert-base-german-cased`: A variant of DistilBERT trained specifically for the German language with a cased vocabulary.

Please note that new variants of DistilBERT may have been introduced since my knowledge cutoff. To get the most up-to-date information on available DistilBERT models, I recommend referring to the Hugging Face Model Hub or the official DistilBERT repository.

## Loading the dataset

Mounting the Google drive Content Folder.
Uploading the file to Content Folder.

In [6]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [7]:
csv_file_path = '/content/Pre-requisite_dataset#2.csv'

In [8]:
from datasets import DatasetDict, Dataset, load_dataset, load_metric
dataset = Dataset.from_csv(csv_file_path)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-ebfed4b13a16bf93/0.0.0. Subsequent calls will reuse this data.


In [ ]:
#def convert_label(example):
#    example['label'] = 1 if example['label'] == 'Yes' else 0
#    return example

In [ ]:
#dataset = dataset.map(convert_label)

To access an actual element, using a random index:

In [9]:
dataset[4]

{'Topic': 'Learning image features using 2d convolutional DBNs',
 'Prerequsites': 'Applications of deep networks',
 'label': 1}

Splitting the Data in Train and Test with 80% and 20% ratio respectively. 

In [10]:
dataset = dataset.train_test_split(test_size=0.2)

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

Showing the overall structure of the Dataset dictonary.

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Topic', 'Prerequsites', 'label'],
        num_rows: 10368
    })
    test: Dataset({
        features: ['Topic', 'Prerequsites', 'label'],
        num_rows: 2592
    })
})

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [12]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [13]:
show_random_elements(dataset["train"])

,Topic,Prerequsites,label
0,Data visualization and feature discovery using deep auto-encoders,Restricted Boltzmann machines (RBMs),0
1,Estimating the kernel parameters,Predictions using noise-free observations,1
2,Sensor fusion with unknown precisions *,Linear discriminant analysis (LDA),0
3,Computational complexity of JTA,SSVMs: a non-probabilistic view,0
4,Sparse linear models,Independent Component Analysis (ICA),1
5,Unscented Kalman filter (UKF),Online parameter learning using recursive least squares,0
6,Nearest shrunken centroids classifier *,Model fitting,0
7,LDA and mPCA,Latent variable models for discrete data,1
8,Gibbs sampling is a special case of MH,The Imputation Posterior (IP) algorithm,1
9,"Bayesian methods for ""fitting"" LG-SSMs",SSM for time series forecasting,1


## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [14]:
from transformers import AutoTokenizer
from transformers import BertTokenizer    

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", use_fast=True)
#tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", use_fast=True)

We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

In [15]:
task_to_keys = {
                "mrpc": ("Topic", "Prerequsites"),
                "rte": ("Topic", "Prerequsites"),
              }

We can double check it does work on our current dataset:

In [16]:
sentence1_key, sentence2_key = task_to_keys[task]
print("sentence1_key :{}, sentence2_key:{}".format(sentence1_key, sentence2_key))
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Topic: {dataset['train'][0][sentence1_key]}")
    print(f"Prerequsites: {dataset['train'][0][sentence2_key]}")

sentence1_key :Topic, sentence2_key:Prerequsites
Topic: Mercer (positive definite) kernels
Prerequsites: RBF kernels


We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model.

In [17]:
def preprocess_function(examples):
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation='do_not_truncate', max_length=100, padding="max_length")

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [18]:
preprocess_function(dataset['train'][:2])

{'input_ids': [[101, 13081, 1006, 3893, 15298, 1007, 16293, 2015, 102, 21144, 2546, 16293, 2015, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 7149, 2007, 4394, 2951, 102, 8446, 1029, 1055, 4118, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [19]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/10368 [00:00<?, ? examples/s]

Map:   0%|          | 0/2592 [00:00<?, ? examples/s]

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about sentence classification, we use the `AutoModelForSequenceClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem.:

In [20]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
#model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

To instantiate a `Trainer`, we will need to define two more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [21]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-Pre_requisite_finder",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay. Since the best model might not be the one at the end of training, we ask the `Trainer` to load the best model it saved (according to `metric_name`) at the end of training.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-mrpc"` or `"huggingface/bert-finetuned-mrpc"`).

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits (our just squeeze the last axis in the case of STS-B):

In [30]:
def write_to_file(filename, data):
  with open(filename, 'w', encoding='utf-8') as file:
    file.write(data)

In [40]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [42]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    #print("Eval", eval_pred)
    #print("predictions : {}, labels : {}".format(predictions, labels))
    predictions = np.argmax(predictions, axis=1)
    accuracy = metric.compute(predictions=predictions, references=labels)
    print ("predictions :{}, accuracy : {}".format(str(predictions), accuracy))
    write_to_file("/content/Predictions.csv", str(predictions.tolist()))
    return (accuracy)

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [23]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [43]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/content/distilbert-base-uncased-finetuned-Pre_requisite_finder is already a clone of https://huggingface.co/satyamverma/distilbert-base-uncased-finetuned-Pre_requisite_finder. Make sure you pull the latest changes with `repo.git_pull()`.


You might wonder why we pass along the `tokenizer` when we already preprocessed our data. This is because we will use it once last time to make all the samples we gather the same length by applying padding, which requires knowing the model's preferences regarding padding (to the left or right? with which token?). The `tokenizer` has a pad method that will do all of this right for us, and the `Trainer` will use it. You can customize this part by defining and passing your own `data_collator` which will receive the samples like the dictionaries seen above and will need to return a dictionary of tensors.

We can now finetune our model by just calling the `train` method:

In [ ]:
training_args = TrainingArguments(
    output_dir="Pre-requisite_Model_2",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/content/Pre-requisite_Model_2 is already a clone of https://huggingface.co/satyamverma/Pre-requisite_Model_2. Make sure you pull the latest changes with `repo.git_pull()`.


In [25]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.694300,0.686561,0.547068
2,0.687900,0.673186,0.554398
3,0.676700,0.670381,0.562886
4,0.644500,0.690887,0.538194
5,0.627400,0.694805,0.555556


predictions :[1 0 1 ... 1 0 0], accuracy : {'accuracy': 0.5470679012345679}
predictions :[0 0 1 ... 1 0 0], accuracy : {'accuracy': 0.5543981481481481}
predictions :[0 0 1 ... 1 0 0], accuracy : {'accuracy': 0.5628858024691358}
predictions :[0 1 1 ... 1 0 1], accuracy : {'accuracy': 0.5381944444444444}
predictions :[0 1 1 ... 1 0 1], accuracy : {'accuracy': 0.5555555555555556}


TrainOutput(global_step=3240, training_loss=0.6623901178807388, metrics={'train_runtime': 609.6834, 'train_samples_per_second': 85.028, 'train_steps_per_second': 5.314, 'total_flos': 1341232411392000.0, 'train_loss': 0.6623901178807388, 'epoch': 5.0})

We can check with the `evaluate` method that our `Trainer` did reload the best model properly (if it was not the last one):

In [44]:
trainer.evaluate()

predictions :[0 0 1 ... 1 0 0], accuracy : {'accuracy': 0.5628858024691358}


{'eval_loss': 0.6703808903694153,
 'eval_accuracy': 0.5628858024691358,
 'eval_runtime': 7.4698,
 'eval_samples_per_second': 346.996,
 'eval_steps_per_second': 21.687}

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/255M [00:00<?, ?B/s]

Upload file runs/May23_08-38-17_f0bd359f2c6e/events.out.tfevents.1684831102.f0bd359f2c6e.394.24:   0%|        …

Upload file runs/May23_08-38-17_f0bd359f2c6e/events.out.tfevents.1684831493.f0bd359f2c6e.394.26:   0%|        …

To https://huggingface.co/satyamverma/Pre-requisite_Model_2
   59aaf9f..ddc6afa  main -> main

   59aaf9f..ddc6afa  main -> main

To https://huggingface.co/satyamverma/Pre-requisite_Model_2
   ddc6afa..d537d17  main -> main

   ddc6afa..d537d17  main -> main



'https://huggingface.co/satyamverma/Pre-requisite_Model_2/commit/ddc6afa65815bf42736768d9ad7d4581ac84e544'

## Hyperparameter search

The `Trainer` supports hyperparameter search using [optuna](https://optuna.org/) or [Ray Tune](https://docs.ray.io/en/latest/tune/). For this last section you will need either of those libraries installed, just uncomment the line you want on the next cell and run it.

In [ ]:
#! pip install optuna
#! pip install ray[tune]

During hyperparameter search, the `Trainer` will run several trainings, so it needs to have the model defined via a function (so it can be reinitialized at each new run) instead of just having it passed. We jsut use the same function as before:

In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

And we can instantiate our `Trainer` like before:

In [ ]:
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

The method we call this time is `hyperparameter_search`. Note that it can take a long time to run on the full dataset for some of the tasks. You can try to find some good hyperparameter on a portion of the training dataset by replacing the `train_dataset` line above by:
```python
train_dataset = encoded_dataset["train"].shard(index=1, num_shards=10) 
```
for 1/10th of the dataset. Then you can run a full training on the best hyperparameters picked by the search.

In [45]:
#best_run = trainer.hyperparameter_search(n_trials=6, direction="maximize")

The `hyperparameter_search` method returns a `BestRun` objects, which contains the value of the objective maximized (by default the sum of all metrics) and the hyperparameters it used for that run.

In [ ]:
best_run

BestRun(run_id='8', objective=0.631768953068592, hyperparameters={'learning_rate': 4.247277359513074e-05, 'num_train_epochs': 3, 'seed': 8, 'per_device_train_batch_size': 32}, run_summary=None)

You can customize the objective to maximize by passing along a `compute_objective` function to the `hyperparameter_search` method, and you can customize the search space by passing a `hp_space` argument to `hyperparameter_search`. See this [forum post](https://discuss.huggingface.co/t/using-hyperparameter-search-in-trainer/785/10) for some examples.

To reproduce the best training, just set the hyperparameters in your `TrainingArgument` before creating a `Trainer`:

In [ ]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classi

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.689807,0.581227
2,No log,0.665415,0.606498
3,No log,0.682738,0.631769


TrainOutput(global_step=234, training_loss=0.6310907999674479, metrics={'train_runtime': 195.6038, 'train_samples_per_second': 38.189, 'train_steps_per_second': 1.196, 'total_flos': 355213916749944.0, 'train_loss': 0.6310907999674479, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub()

Upload file runs/May21_18-24-09_1d185abc4b07/events.out.tfevents.1684695850.1d185abc4b07.310.24: 100%|########…

To https://huggingface.co/satyamverma/distilbert-base-uncased-finetuned-rte
   054e621..300ba30  main -> main

   054e621..300ba30  main -> main

To https://huggingface.co/satyamverma/distilbert-base-uncased-finetuned-rte
   300ba30..8d7894b  main -> main

   300ba30..8d7894b  main -> main



'https://huggingface.co/satyamverma/distilbert-base-uncased-finetuned-rte/commit/300ba3066fd3d00cc53c1b443e450cd02edc4293'